In [5]:
from neurokit.io import Recording
from neurokit.io import read_edf
from neurokit.analysis import suppressions
import neurokit as nk
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

### Reading segments of eeg 

In [6]:
seg1 = read_edf('neurokit/tests/data/EEG_BM65_ies_as.edf')
seg2 = read_edf('neurokit/tests/data/EEG_BM65_none.edf')
seg3 = read_edf('neurokit/tests/data/EEG_BM65_ies.edf')

### Reading reference masks calculated using Matlab code

In [7]:
ref1 = pd.read_csv('neurokit/tests/data/BM65_ies_as.csv')
ref2 = pd.read_csv('neurokit/tests/data/BM65_no_sup.csv')
ref3 = pd.read_csv('neurokit/tests/data/BM65_ies.csv')

### Calculating detections

#### Case 1  : IES detections

In [10]:
intervals1 = suppressions.detect_ies(seg1)
intervals2 = suppressions.detect_ies(seg2)
intervals3 = suppressions.detect_ies(seg3)
pred1 = intervals_to_mask(intervals1, seg1.data.index)
pred2 = intervals_to_mask(intervals2, seg2.data.index)
pred3 = intervals_to_mask(intervals3, seg3.data.index)
perf = pd.DataFrame(columns=['TP', 'TN', 'FP', 'FN', 'accuracy', 'precision'])
perf = perf.append(_calculate_perf_measures(pred1, ref1.loc[:, 'ies']),ignore_index=True)
perf = perf.append(_calculate_perf_measures(pred2, ref2.loc[:, 'ies']),ignore_index=True)
perf = perf.append(_calculate_perf_measures(pred3, ref3.loc[:, 'ies']),ignore_index=True)
perf

,TP,TN,FP,FN,accuracy,precision
0,13913.0,15380.0,11551.0,747.0,0.704311,0.546379
1,2810.0,12595.0,10776.0,286.0,0.582046,0.206831
2,19292.0,11343.0,6300.0,875.0,0.810235,0.753829


#### Case 2 : alpha suppression detection

In [11]:
intervals1 = suppressions.detect_alpha_suppressions(seg1)
intervals2 = suppressions.detect_alpha_suppressions(seg2)
intervals3 = suppressions.detect_alpha_suppressions(seg3)
pred1 = intervals_to_mask(intervals1, seg1.data.index)
pred2 = intervals_to_mask(intervals2, seg2.data.index)
pred3 = intervals_to_mask(intervals3, seg3.data.index)
perf = pd.DataFrame(columns=['TP', 'TN', 'FP', 'FN', 'accuracy', 'precision'])
perf = perf.append(_calculate_perf_measures(pred1, ref1.loc[:, 'as']),ignore_index=True)
perf = perf.append(_calculate_perf_measures(pred2, ref2.loc[:, 'as']),ignore_index=True)
perf = perf.append(_calculate_perf_measures(pred3, ref3.loc[:, 'as']),ignore_index=True)
perf

,TP,TN,FP,FN,accuracy,precision
0,2210.0,3.0,39378.0,0.0,0.053209,0.053140
1,3240.0,27.0,23200.0,0.0,0.123437,0.122542
2,0.0,16.0,37794.0,0.0,0.000423,0.000000


#### Helper functions

In [8]:
def _calculate_perf_measures(prediction, reference):
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    for i in range(len(prediction)):
        if (prediction[i] == reference[i]) and prediction[i]:
            TP+=1
        elif (prediction[i] == reference[i]) and not prediction[i]:
            TN+=1
        elif (prediction[i] != reference[i]) and prediction[i]:
            FP+=1
        else:
            FN+=1
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/ (TP+FP)
    return {'TP': TP, 'TN': TN, 'FP': FP, 'FN': FN, 'accuracy': accuracy, 'precision': precision}
            

In [9]:
def intervals_to_mask(intervals, index):
    mask = np.zeros(len(index), dtype=bool)

    for start, end in intervals.loc[:,['start', 'end']].values:
        mask[index.get_loc(start):index.get_loc(end)] = True

    return mask